In [1]:
import cobra

/Users/helenscott/opt/miniconda3/envs/med4-hot1a3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [41]:
import modelmaker_dilution

In [2]:
model = cobra.io.read_sbml_model("model.xml")

'' is not a valid SBML 'SId'.


In [44]:
model = modelmaker_dilution.add_dilution_reactions(model)
model = modelmaker_dilution.add_dilution_constraints(model, 1000)

The given_model passed to add_dilution_reactions appears to already have at least one dilution reaction; returning the model as-is
